# ANLI Baseline with LLM

You have to implement in this notebook a baseline for ANLI classification using an LLM.
This baseline must be implemented using DSPy.



In [ ]:
# Configure the DSPy environment with the language model - for grok the parameters must be:
# env variable should be in os.environ['XAI_API_KEY']
# "xai/grok-3-mini"
# "gemini/gemini-2.5-flash"
import os
import dspy

In [21]:
with open("grok_key.ini") as f:
        for line in f:
            if "XAI_API_KEY" in line and not line.strip().startswith("#"):
                key_value = line.strip().split("=")
                if len(key_value) == 2:
                    os.environ["XAI_API_KEY"] = key_value[1].split()[0]

with open("gemini_key.ini") as f:
        for line in f:
            if "GEMINI_API_KEY" in line and not line.strip().startswith("#"):
                key_value = line.strip().split("=")
                if len(key_value) == 2:
                    os.environ["GEMINI_API_KEY"] = key_value[1].split()[0]

In [48]:


lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
# for ollama 
# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [69]:
from typing import Literal

#defining the module
class anli_classification_signature(dspy.Signature):

    """Lable the relationship between given premise and hypothesis."""
    
    premise: str = dspy.InputField()
    hypothesis: str = dspy.InputField()
    label: Literal['entailment', 'contradiction', 'neutral'] = dspy.OutputField()
    reason: str = dspy.OutputField()

classify = dspy.ChainOfThought(anli_classification_signature)


## Load ANLI dataset

In [32]:
from datasets import load_dataset

dataset = load_dataset("facebook/anli")
dataset = dataset.filter(lambda x: x['reason'] != None and x['reason'] != "")

In [33]:
dataset

DatasetDict({
    train_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 2923
    })
    dev_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 4861
    })
    dev_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 13375
    })
    dev_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1200


## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [34]:
from evaluate import load

accuracy = load("accuracy")
precision = load("precision")
recall = load("recall")
f1 = load("f1")


In [35]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [36]:
clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])

{'accuracy': 0.6666666666666666,
 'f1': 0.6666666666666666,
 'precision': 1.0,
 'recall': 0.5}

## Your Turn

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]

In [87]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

label_map = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2
}

#defining metric for evaluation
def binary_metrics(example, prediction, trace=None):
    print(example.label)
   # print(prediction["label"])
    return example.label == label_map[prediction["label"]]

example_trainset = []
for example in dataset['dev_r3']:
    example_trainset.append(dspy.Example(premise=example["premise"], hypothesis=example["hypothesis"], label=example["label"], reason=example["reason"]).with_inputs("premise", "hypothesis"))
    if len(example_trainset) >=50:
        break




In [89]:

#optimizing based on dev-r3

dspy.configure_cache(
    enable_disk_cache=True,
    enable_memory_cache=True,
)

config = dict(max_bootstrapped_demos=4, max_labeled_demos=4, num_candidate_programs=10, num_threads=4)

teleprompter = BootstrapFewShotWithRandomSearch(metric=binary_metrics, **config)
optimized_program = teleprompter.compile(classify, trainset=example_trainset)



Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 10 candidate sets.
Average Metric: 34.00 / 50 (68.0%): 100%|██████████| 50/50 [02:21<00:00,  2.84s/it]

2025/08/05 17:07:06 INFO dspy.evaluate.evaluate: Average Metric: 34 / 50 (68.0%)



New best score: 68.0 for seed -3
Scores so far: [68.0]
Best score so far: 68.0
Average Metric: 30.00 / 50 (60.0%): 100%|██████████| 50/50 [01:45<00:00,  2.11s/it]

2025/08/05 17:08:52 INFO dspy.evaluate.evaluate: Average Metric: 30 / 50 (60.0%)



Scores so far: [68.0, 60.0]
Best score so far: 68.0


...

0


 16%|█▌        | 8/50 [00:53<04:43,  6.75s/it]


0
Bootstrapped 4 full traces after 8 examples for up to 1 rounds, amounting to 8 attempts.
Average Metric: 34.00 / 50 (68.0%): 100%|██████████| 50/50 [01:30<00:00,  1.81s/it]

2025/08/05 17:32:22 INFO dspy.evaluate.evaluate: Average Metric: 34 / 50 (68.0%)



Scores so far: [68.0, 60.0, 68.0, 68.0, 58.0, 60.0, 72.0, 72.0, 64.0, 62.0, 64.0, 68.0, 68.0]
Best score so far: 72.0
13 candidate programs found.


In [ ]:
#next on the agenda: 
#create a list of evaluations - pred/gold labels - on test_r3 ( same as 1.2 baseline)
#combine with list made in 1.2 - both pred labels and reasons
#Compare the results with the baseline and provide agreement metrics between the two models.
#agreement metrics: maybe follow GPT, seems reasonable.